In [22]:
import pandas as pd
import numpy as np
import logging

# notebook only
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)


filename="../data/data.csv"
df = pd.read_csv(filename)
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [23]:
missing = sum(df.isna().sum())
logging.info(f"Missing elements: {missing}")

INFO:root:Missing elements: 0


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3900 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [25]:
cols = df.columns
object_cols = []
for i in cols:
    col = df[i]
    if col.dtype == "object":
        object_cols.append(i)

logging.info(f"The columns which contain objects are: {', '.join(object_cols)}")
logging.info("These columns will be encoded")

INFO:root:The columns which contain objects are: Gender, Item Purchased, Category, Location, Size, Color, Season, Subscription Status, Shipping Type, Discount Applied, Promo Code Used, Payment Method, Frequency of Purchases
INFO:root:These columns will be encoded


In [26]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
logging.info("Encoding.......")
for col in object_cols:
    df[col] = le.fit_transform(df[col])
logging.info("Encoding complete")

INFO:root:Encoding.......
INFO:root:Encoding complete


In [27]:
from sklearn.model_selection import train_test_split
import os
logging.info("")
processed_data = pd.read_csv("../data/processed_data.csv")
target_column = 'Frequency of Purchases'
X = processed_data.loc[:, processed_data.columns != target_column]
y = processed_data.loc[:, processed_data.columns == target_column]
directory = '../data/splits'

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"Directory '{directory}' created successfully!")
else:
    print(f"Directory '{directory}' already exists.")    

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify = y, random_state=47)
    
np.save(f'../data/splits/X_train.npy', X_train)
np.save(f'../data/splits/X_test.npy', X_test)
np.save(f'../data/splits/y_train.npy', y_train)
np.save(f'../data/splits/y_test.npy', y_test)

INFO:root:


Directory '../data/splits' already exists.


In [28]:
# import numpy as np
# import optuna
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# X_train = np.load('../data/splits/X_train.npy')
# y_train = np.load('../data/splits/y_train.npy')


# def objective(trial):
#     params = {
#         'objective': 'reg:squarederror',
#         'eval_metric': 'rmse',
#         'booster': 'gbtree',
#         'lambda': trial.suggest_float('lambda', 1e-8, 1.0),
#         'alpha': trial.suggest_float('alpha', 1e-8, 1.0),
#         'max_depth': trial.suggest_int('max_depth', 3, 10),
#         'eta': trial.suggest_float('eta', 1e-3, 0.5),
#         'gamma': trial.suggest_float('gamma', 1e-8, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'random_state': 42,
#         'n_jobs': -1
#     }

#     X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#     dtrain = xgb.DMatrix(X_train_split, label=y_train_split)
#     dval = xgb.DMatrix(X_val, label=y_val)

#     bst = xgb.train(params, dtrain, num_boost_round=trial.suggest_int('n_estimators', 100, 1000, step=100), evals=[(dval, 'eval')], early_stopping_rounds=10, verbose_eval=False)

#     y_pred = bst.predict(dval)
#     rmse = mean_squared_error(y_val, y_pred, squared=False)
    
#     return rmse

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)

# best_params = study.best_params
# print("Best hyperparameters:", best_params)

In [29]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from aim import Run

X_train = np.load('../data/splits/X_train.npy')
y_train = np.load('../data/splits/y_train.npy')

def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'eta': trial.suggest_float('eta', 1e-3, 0.5),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'random_state': 42,
        'n_jobs': -1
    }

    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    dtrain = xgb.DMatrix(X_train_split, label=y_train_split)
    dval = xgb.DMatrix(X_val, label=y_val)

    bst = xgb.train(params, dtrain, num_boost_round=trial.suggest_int('n_estimators', 100, 1000, step=100), evals=[(dval, 'eval')], early_stopping_rounds=10, verbose_eval=False)

    y_pred = bst.predict(dval)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    run = Run()
    run["hparams"] = {k: v for k, v in params.items() if k != 'eval_metric'}
    run.track(rmse, name='RMSE', context={"subset": "validation"})

    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Best hyperparameters:", best_params)

[I 2023-11-28 17:57:15,832] A new study created in memory with name: no-name-9e3bd0d0-dbc3-4387-9819-ea4092d069c8


DEBUG:filelock:Attempting to acquire lock 140524253163280 on /run/media/veer/613A1DEC519A152F1/kf-pipeline/notebooks/.aim/locks/structured_db_lock
DEBUG:filelock:Lock 140524253163280 acquired on /run/media/veer/613A1DEC519A152F1/kf-pipeline/notebooks/.aim/locks/structured_db_lock
DEBUG:aim.sdk.reporter:time remaining: -10.007867506000025
DEBUG:aim.sdk.reporter:only -10.007867506000025 remaining... flushing one task
DEBUG:aim.sdk.reporter:calibrated check-in: CheckIn(idx=12, expect_next_in=30, flag_name='progress') -> CheckIn(idx=12, expect_next_in=15, flag_name='progress')
DEBUG:aim.sdk.reporter.file_manager:touching check-in: /run/media/veer/613A1DEC519A152F1/kf-pipeline/notebooks/.aim/check_ins/dc64f81bc3c849f3aa8fc9f7-0000000000000012-progress-1701174436.39-00015
DEBUG:aim.sdk.reporter.file_manager:found 0 check-ins:
DEBUG:aim.sdk.reporter.file_manager:the acting one: /run/media/veer/613A1DEC519A152F1/kf-pipeline/notebooks/.aim/check_ins/dc64f81bc3c849f3aa8fc9f7-0000000000000012-pro

Best hyperparameters: {'lambda': 0.7843513260082775, 'alpha': 0.7857700292648848, 'max_depth': 9, 'eta': 0.02880256390533755, 'gamma': 0.5116481138728615, 'colsample_bytree': 0.6506889940432599, 'min_child_weight': 10, 'subsample': 0.991195724423565, 'n_estimators': 800}


In [ ]:
!aim up

Running Aim UI on repo `<Repo#-1019572160593031488 path=/run/media/veer/613A1DEC519A152F1/kf-pipeline/notebooks/.aim read_only=None>`
Open http://127.0.0.1:43800
Press Ctrl+C to exit


DEBUG:aim.sdk.reporter:time remaining: -25.003393958999993
DEBUG:aim.sdk.reporter:only -25.003393958999993 remaining... flushing one task
DEBUG:aim.sdk.reporter:Skipping touching `progress` as it is already up to date
DEBUG:aim.sdk.reporter:progress is not scheduled anymore. It will be invoked as soon as next one appears
DEBUG:aim.sdk.reporter:no interesting things to do, sleeping for 0
DEBUG:aim.sdk.reporter:until woken up
DEBUG:aim.sdk.reporter:unfinished tasks: 0
DEBUG:aim.sdk.reporter:no interesting things to do, sleeping for 30
DEBUG:aim.sdk.reporter:until woken up
DEBUG:aim.sdk.reporter:unfinished tasks: 0
DEBUG:aim.sdk.reporter:no interesting things to do, sleeping for 30
DEBUG:aim.sdk.reporter:until woken up
DEBUG:aim.sdk.reporter:unfinished tasks: 0
DEBUG:aim.sdk.reporter:no interesting things to do, sleeping for 30
DEBUG:aim.sdk.reporter:until woken up
DEBUG:aim.sdk.reporter:unfinished tasks: 0
DEBUG:aim.sdk.reporter:no interesting things to do, sleeping for 30
DEBUG:aim.sdk.

^C


DEBUG:aim.sdk.reporter:no interesting things to do, sleeping for 30
DEBUG:aim.sdk.reporter:until woken up
DEBUG:aim.sdk.reporter:unfinished tasks: 0
DEBUG:aim.sdk.reporter:no interesting things to do, sleeping for 30
DEBUG:aim.sdk.reporter:until woken up
DEBUG:aim.sdk.reporter:unfinished tasks: 0
DEBUG:aim.sdk.reporter:no interesting things to do, sleeping for 30
DEBUG:aim.sdk.reporter:until woken up
DEBUG:aim.sdk.reporter:unfinished tasks: 0
DEBUG:aim.sdk.reporter:no interesting things to do, sleeping for 30
DEBUG:aim.sdk.reporter:until woken up
DEBUG:aim.sdk.reporter:unfinished tasks: 0
DEBUG:aim.sdk.reporter:no interesting things to do, sleeping for 30
DEBUG:aim.sdk.reporter:until woken up
DEBUG:aim.sdk.reporter:unfinished tasks: 0
DEBUG:aim.sdk.reporter:no interesting things to do, sleeping for 30
DEBUG:aim.sdk.reporter:until woken up
DEBUG:aim.sdk.reporter:unfinished tasks: 0
DEBUG:aim.sdk.reporter:no interesting things to do, sleeping for 30
DEBUG:aim.sdk.reporter:until woken up
